# Training Potts Models with Contrastive Divergence for Protein Design

## GREMLIN

https://github.com/whbpt/GREMLIN_PYTORCH/blob/master/GREMLIN_pytorch.ipynb

#### Import

In [ ]:
# IMPORTANT, only tested using PYTHON 3!
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
from scipy import stats
from scipy.spatial.distance import pdist,squareform
import pandas as pd

#### Params

In [ ]:
################
# note: if you are modifying the alphabet
# make sure last character is "-" (gap)
################
alphabet = "ARNDCQEGHILKMFPSTWYV-"
states = len(alphabet)
a2n = {}
for a,n in zip(alphabet,range(states)):
  a2n[a] = n
################

def aa2num(aa):
  '''convert aa into num'''
  if aa in a2n: return a2n[aa]
  else: return a2n['-']

In [ ]:
## Convert FASTA to MSA np.array()

def parse_fasta(filename):
  '''function to parse fasta file'''
  header = []
  sequence = []
  lines = open(filename, "r")
  for line in lines:
    line = line.rstrip()
    if line[0] == ">":
      header.append(line[1:])
      sequence.append([])
    else:
      sequence[-1].append(line)
  lines.close()
  sequence = [''.join(seq) for seq in sequence]
  return np.array(header), np.array(sequence)

def one_hot(msa,states):
  one = np.eye(states)
  return one[msa]

def mk_msa(seqs):
  '''one hot encode msa'''
  
  ################
  alphabet = "ARNDCQEGHILKMFPSTWYV-"
  states = len(alphabet)
  a2n = {}
  for a,n in zip(alphabet,range(states)):
    a2n[a] = n

  def aa2num(aa):
    '''convert aa into num'''
    if aa in a2n: return a2n[aa]
    else: return a2n['-']
  ################
  
  msa = []
  for seq in seqs:
    msa.append([aa2num(aa) for aa in seq])
  msa_ori = np.array(msa)
  return msa_ori, one_hot(msa_ori,states)

In [ ]:
names,seqs = parse_fasta("pfamncamseed.fas.txt")
msa_ori, msa = mk_msa(seqs)

print(msa_ori.shape)
print(msa.shape)

(48, 113)
(48, 113, 21)


In [ ]:
# collecting some information about input msa
N = msa.shape[0] # number of sequences
L = msa.shape[1] # length of sequence
A = msa.shape[2] # number of states (or categories)

In [ ]:
class GREMLIN(torch.nn.Module):
  def __init__(self,L,A):
    super(GREMLIN, self).__init__()
    self.W0 = torch.nn.Parameter(torch.zeros(L*A,L*A), requires_grad=True) # this is J in the manuscript
    self.b0 = torch.nn.Parameter(torch.zeros(L*A), requires_grad=True) # this is H 
    # Use the equation for probability of Boltzmann distribution (without the 1/Z term) to calculate likelihood.
    self.MASK = (1.0 -torch.eye(L*A))
  def forward(self,X):
    X = X.reshape(-1,L*A)
    W = (self.W0+self.W0)/2.0 * self.MASK
   
    MSA_pred = (X.mm(W)+self.b0).reshape(-1,L,A)
    loss = torch.sum(- MSA_Input * F.log_softmax(MSA_pred, -1))
    L2_w = torch.square(W).sum() * 0.01 * 0.5 *L*A
    L2_b = torch.square(self.b0).sum() * 0.01
    loss = loss + L2_w + L2_b
    return loss

In [ ]:
class Model(torch.nn.Module):
  def __init__(self,L,A):
    super(Model, self).__init__()
    self.GREMLIN_ = GREMLIN(L,A)
    
  def forward(self,X):
    loss = self.GREMLIN_(X)
    return loss

In [ ]:
#enviroment setting
device = torch.device("cuda:0") # Uncomment this to run on GPU
MSA_Input = torch.from_numpy(msa.astype(np.float32))

model = Model(L,A)
learning_rate = 0.1*np.log(N)/L
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for t in range(100):

    loss = model(MSA_Input)      
    optimizer.zero_grad()    
    loss.backward()
    optimizer.step()
    
    if (t) % (int(100/10)) == 0: 
      print(t, loss.item())

0 16513.490234375
10 7362.96484375
20 6565.10595703125
30 6130.10791015625
40 5972.81201171875
50 5892.57666015625
60 5842.08740234375
70 5809.5419921875
80 5783.97802734375
90 5762.72412109375


In [ ]:
w = model.GREMLIN_.W0.detach().numpy()
w = (w+w.T).reshape(L,A,L,A)

In [ ]:
model(MSA_Input)  

tensor(5743.6265, grad_fn=<AddBackward0>)

### bmDCA

**Important Notes:**

*  All amino acids must be upper case

https://github.com/ranganathanlab/bmDCA

In [ ]:
!git clone https://github.com/ranganathanlab/bmDCA.git

Cloning into 'bmDCA'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1679 (delta 15), reused 22 (delta 10), pack-reused 1646
Receiving objects: 100% (1679/1679), 790.26 KiB | 10.83 MiB/s, done.
Resolving deltas: 100% (1152/1152), done.
/content/bmDCA
./autogen.sh: 10: ./autogen.sh: autoreconf: not found
/content


In [ ]:
!sudo apt-get update
!sudo apt-get install git gcc g++ automake autoconf pkg-config \
  libarmadillo-dev libopenblas-dev libarpack++2-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]


In [ ]:
%cd bmDCA
!bash autogen.sh --prefix=/usr/local && \
%cd ..

/content/bmDCA
configure.ac:8: installing './config.guess'
configure.ac:8: installing './config.sub'
configure.ac:35: installing './install-sh'
configure.ac:35: installing './missing'
src/Makefile.am: installing './depcomp'
+ ./configure --prefix=/usr/local
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... /bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking whether to enable maintainer-specific portions of Makefiles... yes
checking whether make supports nested variables... (cached) yes
checking for g++... g++
checking whether the C++ compiler works... yes
checking for C++ compiler default output file name... a.out
checking for suffix of executables... 
checking whether we ar

In [ ]:
%%shell
cd bmDCA
make -j4 && \
make install
cd ..

Making all in src
make[1]: Entering directory '/content/bmDCA/src'
  CXX      bmdca.o
  CXX      msa_stats.o
  CXX      model.o
  CXX      msa.o
  CXX      run.o
  CXX      mcmc.o
  CXX      mcmc_stats.o
  CXX      graph.o
  CXX      utils.o
run.cpp: In constructor ‘Sim::Sim(MSAStats, std::__cxx11::string, std::__cxx11::string, bool)’:
run.cpp:383:10: warning: ignoring return value of ‘int chdir(const char*)’, declared with attribute warn_unused_result [-Wunused-result]
     chdir(dest_dir.c_str());
     ~~~~~^~~~~~~~~~~~~~~~~~
  CXX      bmdca_sample.o
  CXX      generator.o
  CXX      arma_convert.o
bmdca_sample.cpp: In function ‘int main(int, char**)’:
bmdca_sample.cpp:124:10: warning: ignoring return value of ‘int chdir(const char*)’, declared with attribute warn_unused_result [-Wunused-result]
     chdir(dest_dir.c_str());
     ~~~~~^~~~~~~~~~~~~~~~~~
  CXX      fasta_convert.o
  CXXLD    bmdca_sample
  CXXLD    bmdca
  CXXLD    arma2ascii
  CXXLD    numeric2fasta
make[1]: Leaving

In [ ]:
!mkdir results

In [ ]:
!cp pfam_hits.txt lcc.fasta

#### Training

100-245 of LCC?

In [ ]:
import numpy as np

def read_fasta(fname):
    seqs = []
    s = ""
    with open(fname) as f:
        line = f.readline()
        while line:
            if line.startswith(">"):
                if s != "":
                    seqs.append(list(s))
                s = ""
            elif len(line) > 0:
                s += line.strip()
            line = f.readline()
        seqs.append(list(s))
    return np.array(seqs)

In [ ]:
seqs = read_fasta("pfam_hits.txt")

In [ ]:
mask = np.zeros(len(seqs[0]), dtype=np.bool)
for i in range(len(seqs[0])):
    gaps = 0
    for s in seqs:
        if s[i] == '-':
            gaps += 1
    if gaps/len(seqs) < 0.67:   # keep positions where less that 2/3rd are gaps
        mask[i] = True
seqs = seqs[:,mask]

In [ ]:
towrite = ""
for i in range(len(seqs)):
    towrite += ">{}\n".format(i)
    towrite += "".join(seqs[i][100:]) + "\n"   # take positions 100-226
with open("lcc_short.fasta",'w') as f:
    f.write(towrite)

In [ ]:
%%shell
rm results/*
bmdca -i lcc_short.fasta -r -d /content/results

745 sequences
226 positions
21 amino acids (including gaps)
682.917 effective sequences
initializing run... 5.03415 sec

Step: 1
sampling model with mcmc... 7.5798 sec
updating mcmc with samples... 4.68259 sec
computing sequence energies and correlations... 0.372186 sec
computing mcmc 1p and 2p statistics... 2.41972 sec
computing error and updating gradient... 1.49729 sec
update learning rate... 0.218366 sec
updating parameters... 0.203105 sec

Step: 2
sampling model with mcmc... 7.49869 sec
updating mcmc with samples... 4.65871 sec
computing sequence energies and correlations... 0.376457 sec
computing mcmc 1p and 2p statistics... 2.45086 sec
computing error and updating gradient... 1.48593 sec
update learning rate... 0.220486 sec
updating parameters... 0.213628 sec

Step: 3
sampling model with mcmc... 7.40256 sec
updating mcmc with samples... 4.64783 sec
computing sequence energies and correlations... 0.371309 sec
computing mcmc 1p and 2p statistics... 2.45801 sec
computing error and 

CalledProcessError: ignored

In [ ]:
!tar -czf boltzmann.tar.gz results/*

#### Sampling

Change temperature in a config file

In [ ]:
%%shell
bmdca_sample -p parameters.txt -d /content/results -o samples.txt -c config.conf

In [ ]:
!perl convert.pl lcc_pfam.txt lcc_pfam.fa


Error: txt is not a valid input format option


### Contrastive Divergence

In [ ]:
import jax.numpy as jnp
from jax import random
from jax import grad
from jax.scipy.stats.norm import pdf
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

key = random.PRNGKey(0)

### Model evaluation

In [ ]:
!git clone https://github.com/igemto-drylab/CSBERG-ML.git
%cd CSBERG-ML
from util import *